In [1]:
## Importing Reqired Libraries

In [2]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import csv
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cv2
import numpy as np
# Loading the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Creating a variable to store face data
individuals = []

def capture_faces(name):
    cap = cv2.VideoCapture(0)
    face_data = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
        
        if len(faces) > 0:
            # Sorting faces by size (largest first)
            faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
            (x, y, w, h) = faces[0]
            
            # Extracting the face region and resize it
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (100, 100))
            
            # Save every 3rd frame
            if frame_count % 3 == 0:
                face_data.append(face)
                print(f"Saved frame {frame_count // 3 + 1}")
            
            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        cv2.imshow("Face Capture", frame)
        
        frame_count += 1

        # Break the loop after capturing 20 frames
        if frame_count >= 18:  # 20 frames saved every 3rd frame
            break

        # Press 'q' to break the loop early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Convert face data to numpy array and save
    face_data = np.asarray(face_data)
    np.save(f'{name}.npy', face_data)
    print(f"Collected {len(face_data)} face images and saved as {name}.npy")

while True:
    name = input('Enter your name: ').capitalize()
    individuals.append(name)
    capture_faces(name)
    
    a = input('Do you want to add another student? (Yes/No): ').capitalize()
    if a == 'No':
        break


Enter your name: Indhu
Saved frame 1
Saved frame 3
Saved frame 5
Saved frame 6
Collected 4 face images and saved as Indhu.npy
Do you want to add another student? (Yes/No): No


In [4]:
# List of individuals to be loaded
individuals = ['Indhu']  # Add the names you have captured

# Loading face data from numpy files
face_data = []
labels = []
label_map = {}  # Map individual names to numerical labels
label_counter = 0

In [5]:
for person in individuals:
    data = np.load(f'{person}.npy')
    print(f"Loaded {person}.npy with shape {data.shape}")  # Print the shape of each loaded array
    
    # Skip empty arrays
    if data.shape[0] == 0:
        print(f"Skipping {person}.npy due to empty data.")
        continue
    
    # Ensure data has the shape (num_images, height, width, channels)
    if len(data.shape) == 3:  # Missing channel dimension
        data = np.expand_dims(data, axis=-1)
    if data.shape[-1] == 1:  # Grayscale images, convert to 3 channels
        data = np.concatenate([data, data, data], axis=-1)
    
    print(f"Processed {person}.npy with shape {data.shape}")  # Print the shape after processing
    
    face_data.append(data)
    labels.append([label_counter] * len(data))
    label_map[label_counter] = person
    label_counter += 1

Loaded Indhu.npy with shape (4, 100, 100, 3)
Processed Indhu.npy with shape (4, 100, 100, 3)


In [6]:
# Concatenate face_data into a single numpy array
if face_data:
    face_data = np.concatenate(face_data, axis=0)
else:
    raise ValueError("No valid data found in face_data. Check input files.")

# Normalize face data
face_data = face_data / 255.0

# Converting labels to categorical format
labels = np.concatenate(labels, axis=0)
labels = to_categorical(labels, num_classes=len(individuals))

In [7]:
# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(face_data, labels, test_size=0.2, random_state=42)

In [8]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(individuals), activation='softmax')
])

In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 1.0

In [11]:
# Save the trained model
model.save('face_recognition_model.h5')

In [12]:
# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture
cap = cv2.VideoCapture(0)

# Open a CSV file to record attendance
with open('attendance.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Time"])

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (100, 100))
            face = face / 255.0
            face = np.expand_dims(face, axis=0)

            # Predict the face
            prediction = model.predict(face)
            label_index = np.argmax(prediction)
            person_name = label_map[label_index]

            # Draw rectangle around the face and label it
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Record attendance
            writer.writerow([person_name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
            print(f"Recorded attendance for {person_name}")

        cv2.imshow("Attendance System", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Recorded attendance for Indhu
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Recorded 